In [1]:
import pandas as pd
from google.cloud import bigquery

In [2]:
country_id = 1

products_query = f"""
    SELECT * 
    FROM `peya-food-and-groceries.user_rodrigo_benitez.product_categories`
    WHERE country_id = {country_id}
    """

client = bigquery.Client()
products_ds = client.query(products_query).to_dataframe()

/home/rodrigobenitez/anaconda3/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/rodrigobenitez/anaconda3/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information 

In [ ]:
# TO CSV

In [3]:
products_ds.sample(5)

,partner_id,partner_name,partner_description,business_type_id,business_type_name,country_id,legacyId_section,section_name,product_legacy_id,product_id,product_name,product_description,gtin,category_level_1_2
253444,377969,Iberpark - Las Piedras Market,None,2,Market,1,5580374,Copetín,98277491,98277491,Snacks Stickletti Y Brezli Chio 250Gr,,04000522061592,Snacks | Salty Snacks
164744,299756,Los Nenes,None,2,Market,1,3613170,Despensa,63401928,63401928,Sopa Instantánea Nissin Sabor Carne Ramen 85 g,,07891079012215,Packaged Foods | Canned / Jarred / Instant Meals
222789,286800,PedidosYa Market 12,None,2,Market,1,5473631,Cuidado menstrual,67952408,67952408,Toallas Femeninas Siempre Libre Especial Adapt...,,07790010569538,Personal Care / Baby / Health | Personal Care ...
25294,286800,PedidosYa Market 12,None,2,Market,1,5473590,Insecticidas,78103323,78103323,Líquido eléctrico advanced 45 noches raid,,07790520009784,Home / Pet | Household
221968,313420,PedidosYa Market 42,None,2,Market,1,5473440,Panes envasados,78096692,78096692,Pan Integral de Centeno Mestemacher con Linaza...,,04000446001780,Bread / Bakery | Bread


In [ ]:
model_info = products_ds[['product_id', 'section_name', 'product_name', 'product_description']].fillna('')